Ref: https://github.com/SchindlerLiang/VAE-for-Anomaly-Detection

In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [4]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=1
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [5]:
def LSTM_VAE(X_train):
    latent_dim = 16

    encoder_inputs = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = layers.LSTM(64, return_sequences=True)(encoder_inputs)
    x = layers.LSTM(32)(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.RepeatVector(X_train.shape[1])(latent_inputs)
    x = layers.LSTM(32, return_sequences=True)(x)
    x = layers.LSTM(64)(x)
    x = layers.Dense(X_train.shape[1] *  X_train.shape[2])(x)
    decoder_outputs = layers.Reshape([X_train.shape[1], X_train.shape[2]])(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

    model = VAE(encoder, decoder)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
    history = model.fit(X_train, epochs=50, batch_size=128, verbose=0)
    return model

### Yahoo S5

In [6]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [7]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(8, 4)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

yahoo_A1 0.9999999000000052 0.4999999375000063 0.9999999490291289
yahoo_A1 0.9999999166666703 0.6666666055555603 0.9999999656957941
yahoo_A1 0.9090908429752097 0.7547169388267891 0.9166666504722225
yahoo_A1 0.9999998500000123 0.0 0.9999998990476291
yahoo_A1 0.666666577777783 0.007758805445957393 0.5432692030833964
yahoo_A1 0.666666577777783 0.03041666502243064 0.889423031596711
yahoo_A1 0.9999999166666703 0.6666666055555603 0.999999965714287
yahoo_A1 0.7234042060660961 0.7089767156305357 0.7599540193970016
yahoo_A1 0.8679244758988992 0.8540788660511682 0.902444440233037
yahoo_A1 0.999999930000003 0.7999999503333362 0.9999999790099013
yahoo_A1 0.4999999500000025 0.23824418055854407 0.5632996509652418
yahoo_A1 0.999999930000003 0.7999999503333362 0.9999999790000004
yahoo_A1 0.9999999437500025 0.937499976331382 0.9999999926264045
yahoo_A1 0.05555555005144069 0.005154639776318988 0.05882352347174227
yahoo_A1 0.7999999340000034 0.5077777543339514 0.9844659987723635
yahoo_A1 0.99999994090909

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/92 [00:00<?, ?it/s]

yahoo_A3 0.03174602857142873 0.00416666685300557 0.024590161455254233
yahoo_A3 0.12121210964187425 0.029655437819469395 0.2575533611747186
yahoo_A3 0.2580644832466217 0.09280114007960671 0.7443502694492327
yahoo_A3 0.03174602857142873 0.004065040847075319 -0.0
yahoo_A3 0.4999999450000051 0.2702172322004872 0.735260016314079
yahoo_A3 0.0624999938476567 0.013522201907428434 0.36666665115277847
yahoo_A3 0.17647057188581453 0.05112110153369423 0.338082833107246
yahoo_A3 0.12903224281998069 0.03649227058075802 0.5146186324865523
yahoo_A3 0.2105262703601196 0.04853750132036813 0.48036721762680795
yahoo_A3 0.16666662638889695 0.0694255086569315 0.4362016092407416
yahoo_A3 0.16666663611111507 0.08196546419518101 0.5059245269925217
yahoo_A3 0.14925371730897866 0.0425578164818827 0.41666665963514116
yahoo_A3 0.12499997656250315 0.034282261752450444 0.7663934036750893
yahoo_A3 0.17647057188581453 0.04114450459468516 0.2663046242401807
yahoo_A3 0.04724408980097991 0.004098361322217882 -0.0
yahoo_A

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

yahoo_A4 0.11764702906575088 0.04152402124251749 0.5254237173372118
yahoo_A4 0.03571427869898062 0.010803409575720794 0.454917994504169
yahoo_A4 0.06451610613944557 0.01735303456204603 0.3791666519409729
yahoo_A4 0.07017543157894772 0.01872053832530815 0.574999970770835
yahoo_A4 0.33333328333333917 0.14915382578283082 0.769319569083057
yahoo_A4 0.8571427959183704 0.6507221957226789 0.8483682142600166
yahoo_A4 0.16666661944445615 0.045581665669886445 0.5409722057008107
yahoo_A4 0.21428568341837106 0.050983698583095365 0.5680672143797756
yahoo_A4 0.1599999622400081 0.051997672565854205 0.5488700454830671
yahoo_A4 0.11111109074074393 0.03474461219092204 0.5833333175694448
yahoo_A4 0.12307691133727908 0.04786559109971808 0.4343955188696134
yahoo_A4 0.1999999480000115 0.04442413622614971 0.5472222033171303
yahoo_A4 0.0624999938476567 0.019250043602170877 0.4736110920127323
yahoo_A4 0.14814811687243384 0.054336684316932476 0.5838982930461316
yahoo_A4 0.03174602857142873 0.004132231594798262 

In [8]:
yahoo_results = pd.DataFrame(total_scores)
yahoo_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
yahoo_A1,0.647689,0.396152,0.780687
yahoo_A2,0.680355,0.359225,0.869629
yahoo_A3,0.219322,0.098071,0.533683
yahoo_A4,0.205947,0.079319,0.515796


### NASA

In [9]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [10]:
for loader in [load_nasa]:
    datasets = loader(8, 4)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

D1 0.2295534087061448 0.10793226407880574 0.4010378124729783
D2 0.19498578202833744 0.5213298008524826 0.513157749992185


In [11]:
nasa_results = pd.DataFrame(total_scores)
nasa_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.229553,0.107932,0.401038
D2,0.194986,0.521330,0.513158


### SMD

In [12]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [13]:
for loader in [load_smd]:
    datasets = loader(8, 4)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

smd 0.17627464443920743 0.5532991977145021 0.5087209301507045
smd 0.575438546358884 0.477979603052411 0.9088020183604875
smd 0.14329452657629374 0.11290868440813018 0.6579275376689508
smd 0.19889498745956366 0.13637316105845054 0.6385503080949293
smd 0.6551723644470901 0.6210868635302293 0.9003226329057888
smd 0.8375575543931467 0.8371452963774362 0.8935900921303914
smd 0.1906216415238548 0.1605218035454039 0.5203745887956518
smd 0.26562497675781443 0.21589174982730974 0.7248576134191864
smd 0.43613702154628337 0.3722999764576605 0.7439982590400577
smd 0.21837868410447125 0.2072482321212552 0.5095051127977044
smd 0.5222929433080497 0.5629247337277297 0.7391827592676893
smd 0.31598510338787694 0.34192238894161 0.7875229434166391
smd 0.2598424700291495 0.24696097042522702 0.7787462600088423
smd 0.4585986888393067 0.3793011378802644 0.6570459735071392
smd 0.6814158814550898 0.6012775845245948 0.8885580815097753
smd 0.9499999477500025 0.8942184364549642 0.9760487986827799
smd 0.52229295397

In [14]:
smd_results = pd.DataFrame(total_scores)
smd_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
smd,0.441956,0.438788,0.750786


### ECG

In [15]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [16]:
for loader in [load_ecg]:
    datasets = loader(4, 2)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

D1 0.4374999582248303 0.4397686314890952 0.6147846248272459
D2 0.3432835337603096 0.30498595694648606 0.637751424022442
D3 0.1732283043834189 0.06129554680489187 0.5218886023672688
D4 0.2697946900319091 0.16137505821838194 0.6297597591037961
D5 0.3636363172686519 0.23956424399811937 0.7822901846032747
D6 0.1600688316301364 0.08727833173966326 0.47376639360118794
D7 0.05630565370964904 0.019636812864845515 0.5370151946718661
D8 0.17631223154403106 0.0791394892781095 0.4371698030206965
D9 0.3580130914399724 0.15941049794538942 0.35057124961128805


In [17]:
ecg_results = pd.DataFrame(total_scores)
ecg_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.437500,0.439769,0.614785
D2,0.343284,0.304986,0.637751
D3,0.173228,0.061296,0.521889
D4,0.269795,0.161375,0.629760
D5,0.363636,0.239564,0.782290
D6,0.160069,0.087278,0.473766
D7,0.056306,0.019637,0.537015
D8,0.176312,0.079139,0.437170
D9,0.358013,0.159410,0.350571


### Power Demand

In [18]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [19]:
for loader in [load_power_demand]:
    datasets = loader(16, 8)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc']) 

  0%|          | 0/1 [00:00<?, ?it/s]

power_demand 0.26431715744532397 0.10591431534015816 0.36459359455697926


In [20]:
power_results = pd.DataFrame(total_scores)
power_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
power_demand,0.264317,0.105914,0.364594


### 2D Gesture

In [21]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [22]:
for loader in [load_gesture]:
    datasets = loader(4, 2)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/1 [00:00<?, ?it/s]

gesture 0.42618738390451827 0.2573352531304127 0.5187790930544846


In [23]:
gesture_results = pd.DataFrame(total_scores)
gesture_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
gesture,0.426187,0.257335,0.518779
